In [1]:
import ophion
import json
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas
import math

In [2]:
%matplotlib inline

In [3]:
O = ophion.Ophion("http://bmeg.io")

In [4]:
drugs = list(set(O.query().has("gid", "cohort:CCLE").outgoing("hasMember").incoming("responseOf").outgoing("responseTo").values(["gid"]).execute()['result']))

In [5]:
GENE="CCND1"

In [6]:
res = O.query().has("gid", "gene:%s" % (GENE)).incoming("calledInGene").mark("a").outgoing("inCNACallSet").\
outgoing("cnaCallsFor").mark("b").incoming("hasMember").has("gid", "cohort:CCLE").select(["a", "b"]).execute()['result']


In [7]:
positive = set()
negative = set()
for row in res:
    if 'value' in row['a']['properties']:
        cna = math.pow(2, row['a']['properties']['value'] ) * 2.0
        if cna >= 2.6:
            positive.add(row['b']['properties']["gid"])
        else:
            negative.add(row['b']['properties']["gid"])

In [8]:
print len(negative)
print len(positive)

781
235


In [9]:
def get_set_drug_summaries(samples, drug, responseType):
    #data = O.query().has("gid", samples).incoming("responseOf").mark("a").outgoing("responseTo").\
    #has("gid", drug).select("a").values(["gid", "responseSummary"]).execute()['result']
    data = O.query().has("gid", drug).incoming("responseTo").mark("a").outgoing("responseOf").has("gid", samples).select("a").values(["gid", "responseSummary"]).execute()['result']
    data_map = dict(list( (data[i], json.loads(data[i+1])) for i in range(0, len(data), 2) ))
    #print data_map
    values = list( filter(lambda x:x["type"] == responseType, i) for i in data_map.values())
    values = filter(lambda x: len(x) > 0, values)
    values = map(lambda x:x[0]['value'], values)
    return values

In [10]:
p = get_set_drug_summaries(list(positive), drugs[0], "AUC")
n = get_set_drug_summaries(list(negative), drugs[0], "AUC")

In [11]:
#drug_res_pos = O.query().has("gid", list(positive)).mark("a").incoming("responseOf").mark("b").outgoing("responseTo").mark("c").select(["a", "b", "c"]).execute()['result']

In [12]:
#drug_res_neg = O.query().has("gid", list(negative)).mark("a").incoming("responseOf").mark("b").outgoing("responseTo").mark("c").select(["a", "b", "c"]).execute()['result']

In [ ]:
"""
responses = {}
for d in drug_res:
    sample = d['a']['properties']['gid']
    drug = d['c']['properties']['gid']
    o = list(i['value'] for i in d['b']['properties']['summary'] if i['type'] == "AUC")
    if len(o):
        if drug not in responses:
            responses[drug] = {}
        responses[drug][sample] = o[0]
"""

'\nresponses = {}\nfor d in drug_res:\n    sample = d[\'a\'][\'properties\'][\'gid\']\n    drug = d[\'c\'][\'properties\'][\'gid\']\n    o = list(i[\'value\'] for i in d[\'b\'][\'properties\'][\'summary\'] if i[\'type\'] == "AUC")\n    if len(o):\n        if drug not in responses:\n            responses[drug] = {}\n        responses[drug][sample] = o[0]\n'

In [ ]:
out = []
for drug in drugs:
    print drug
    p = get_set_drug_summaries(list(positive), drug, "AUC")
    n = get_set_drug_summaries(list(negative), drug, "AUC")
    s = stats.ttest_ind(p, n, equal_var=False)
    out.append( {"drug" : drug, "statistic" : s.statistic, "pvalue" : s.pvalue} )

compound:JQ-1:MK-0752 (1:1 mol/mol)
compound:A-804598
compound:CIL55A
compound:I-BET151
compound:SNX-2112
compound:NSC95397
compound:palmostatin B
compound:DBeQ
compound:JW-55
compound:tivantinib
compound:Compound 7d-cis
compound:importazole
compound:pifithrin-mu
compound:AA-COCF3
compound:ciclosporin
compound:BRD-K48477130
compound:SR8278
compound:sitagliptin
compound:QW-BI-011
compound:tipifarnib-P1
compound:tipifarnib-P2
compound:tanespimycin:bortezomib (250:1 mol/mol)
compound:salermide
compound:BRD-A05715709
compound:neratinib
compound:dinaciclib
compound:ibrutinib
compound:docetaxel
compound:BRD-K19103580
compound:BRD4132
compound:BRD-K58730230
compound:UNC0638:navitoclax (1:1 mol/mol)
compound:cediranib
compound:marinopyrrole A
compound:necrosulfonamide
compound:HC-067047
compound:CD-1530
compound:KPT185
compound:GSK461364
compound:BRD-K92856060
compound:N9-isopropylolomoucine
compound:CIL56
compound:GDC-0879
compound:navitoclax:sorafenib (1:1 mol/mol)
compound:BRD-A02303741
com

In [ ]:
pandas.DataFrame(out).sort_values("statistic")